In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split

%load_ext autoreload
%autoreload 2

# making all pandas columns visable with display command
pd.set_option('display.max_columns', 500)

In [2]:
# df_EPL = pd.read_csv('datasets/EPL.csv', index_col = 0)
df_EPL = pd.read_csv('datasets/EPL_sort.csv', index_col = 0)
df_EPL

,id,season,stage,date,match_api_id,home_team_api_id,away_team_api_id,home_team_goal,away_team_goal,B365H,B365D,B365A,home_y_card,home_r_card,away_y_card,away_r_card,home_foulcommit,away_foulcommit,home_shoton,away_shoton,home_shotoff,away_shotoff,home_cross,away_cross,home_corner,away_corner,home_possession,away_possession,HTP,ATP,Result
1,1730,2008/2009,1,2008-08-16 00:00:00,489043,Arsenal,West Bromwich Albion,1,0,1.20,6.50,15.00,0,0,0,0,11,9,12,2,13,3,21,7,7,5,66,34,0,0,H
7,1736,2008/2009,1,2008-08-16 00:00:00,489049,Bolton Wanderers,Stoke City,3,1,1.83,3.50,4.50,1,0,2,0,14,15,5,1,7,6,22,29,4,3,47,53,0,0,H
5,1734,2008/2009,1,2008-08-16 00:00:00,489047,Everton,Blackburn Rovers,2,3,2.00,3.30,4.00,2,0,2,0,11,11,2,8,7,8,14,21,3,4,51,49,0,0,A
8,1737,2008/2009,1,2008-08-16 00:00:00,489050,Hull City,Fulham,2,1,2.60,3.20,2.80,3,0,0,0,10,9,5,4,5,6,16,14,5,6,53,47,0,0,H
6,1735,2008/2009,1,2008-08-16 00:00:00,489048,Middlesbrough,Tottenham Hotspur,2,1,3.20,3.40,2.25,1,0,2,0,11,11,7,2,5,5,25,27,7,9,53,47,0,0,H
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2976,4705,2015/2016,38,2016-05-15 00:00:00,1987603,Stoke City,West Ham United,2,1,3.50,3.60,2.15,3,0,1,0,8,12,3,10,2,7,13,18,4,10,45,55,48,62,H
2977,4706,2015/2016,38,2016-05-15 00:00:00,1987604,Swansea City,Manchester City,1,1,6.00,4.75,1.53,1,0,3,0,5,19,1,5,2,12,14,15,4,4,50,50,46,65,D
2978,4707,2015/2016,38,2016-05-15 00:00:00,1987605,Watford,Sunderland,2,2,2.05,3.75,3.70,1,0,0,0,10,9,6,2,10,2,9,12,7,4,49,51,44,38,D
2979,4708,2015/2016,38,2016-05-15 00:00:00,1987606,West Bromwich Albion,Liverpool,1,1,2.40,3.60,3.00,3,0,1,0,12,13,5,3,8,3,12,14,7,5,33,67,42,59,D


In [3]:
df_EPL['Result']

1       H
7       H
5       A
8       H
6       H
       ..
2976    H
2977    D
2978    D
2979    D
2973    H
Name: Result, Length: 3040, dtype: object

In [21]:
X = df_EPL.drop(columns = ['Result','season','date','home_team_api_id','away_team_api_id'])
y_cat = pd.Categorical(df_EPL['Result'])
y = y.codes

In [22]:
print(X,y)

        id  stage  match_api_id  home_team_goal  away_team_goal  B365H  B365D  \
1     1730      1        489043               1               0   1.20   6.50   
7     1736      1        489049               3               1   1.83   3.50   
5     1734      1        489047               2               3   2.00   3.30   
8     1737      1        489050               2               1   2.60   3.20   
6     1735      1        489048               2               1   3.20   3.40   
...    ...    ...           ...             ...             ...    ...    ...   
2976  4705     38       1987603               2               1   3.50   3.60   
2977  4706     38       1987604               1               1   6.00   4.75   
2978  4707     38       1987605               2               2   2.05   3.75   
2979  4708     38       1987606               1               1   2.40   3.60   
2973  4702     38       1987600               3               1   1.67   4.20   

      B365A  home_y_card  h

In [23]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [24]:
print('Train data shape: ', X_train.shape)
print('Train labels shape: ', y_train.shape)
print('Test data shape: ', X_test.shape)
print('Test labels shape: ', y_test.shape)

Train data shape:  (2280, 26)
Train labels shape:  (2280,)
Test data shape:  (760, 26)
Test labels shape:  (760,)


In [37]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, Activation, Flatten, Dense
from tensorflow.keras.optimizers import Adam

# We first need to define the sequence of dependencies (the computational graph)
x = Input(shape=(26,))
# h = Conv2D(32, (7, 7), strides=(2, 2))(x)
# h = Activation('relu')(x)
# h = Flatten()(h)
h = Dense(16)(x)
p = Activation('softmax')(h)

# Now that we have defined how to find p from x, we can create a 
# model simply by saying what is input and what is output
model = Model(inputs=x, outputs=p)
model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam(lr=0.00001), metrics=['accuracy'])
model.summary()

Model: "model_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_10 (InputLayer)        [(None, 26)]              0         
_________________________________________________________________
dense_15 (Dense)             (None, 16)                432       
_________________________________________________________________
activation_16 (Activation)   (None, 16)                0         
Total params: 432
Trainable params: 432
Non-trainable params: 0
_________________________________________________________________


In [38]:
history = model.fit(X_train, y_train, batch_size=64, epochs=5, validation_split=0.04)

Train on 2188 samples, validate on 92 samples
Epoch 1/5
2188/2188 [==============================] - 1s 250us/sample - loss: 601530.5662 - accuracy: 0.0000e+00 - val_loss: 619911.7745 - val_accuracy: 0.0000e+00
Epoch 2/5
2188/2188 [==============================] - 0s 35us/sample - loss: 600716.2331 - accuracy: 0.0000e+00 - val_loss: 619084.1549 - val_accuracy: 0.0000e+00
Epoch 3/5
2188/2188 [==============================] - 0s 36us/sample - loss: 599897.3434 - accuracy: 0.0000e+00 - val_loss: 618253.8424 - val_accuracy: 0.0000e+00
Epoch 4/5
2188/2188 [==============================] - 0s 37us/sample - loss: 599080.3813 - accuracy: 0.0000e+00 - val_loss: 617420.4620 - val_accuracy: 0.0000e+00
Epoch 5/5
2188/2188 [==============================] - 0s 42us/sample - loss: 598263.0166 - accuracy: 0.0000e+00 - val_loss: 616591.4185 - val_accuracy: 0.0000e+00
